In [ ]:
import sys
#sys.path.insert(0, './..')
sys.path.insert(0, './data')

import matplotlib.pyplot as plt
import numpy as np
import plots as pl
from models import model
import torch
from utils import dev, load_data, classification
from torchvision import datasets, transforms
from models import eval

# Make evenly distributed dataset with true samples only

In [ ]:
images, labels = load_data(10000, random=False, train=False)
correct_classes = np.full(labels.shape, True)

for model_id in range(5):
    md = model.madry_diff()
    md.load_state_dict(torch.load('./models/natural_'+str(model_id)+'.pt', map_location=torch.device(dev())))
    md.to(dev())
    md.eval()
    classes = md(images).argmax(1)
    correct_classes[~(classes==labels).cpu()]=False

for model_id in range(5):
    md = model.madry_diff()
    md.load_state_dict(torch.load('./models/robust_'+str(model_id)+'.pt', map_location=torch.device(dev())))
    md.to(dev())
    md.eval()
    classes = md(images).argmax(1)
    correct_classes[~(classes==labels).cpu()]=False

images = images[correct_classes]
labels = labels[correct_classes]

data = {
    'images': images.cpu(),
    'labels': labels.cpu()
}

save_path = '/home/bethge/dschultheiss/AdversarialDecomposition/data/MNIST/stable_data.npy'
# np.save(save_path, data)

# Concatenate data

In [ ]:
model_id = 3
n_batches = 25
model_type = 'natural'

data = np.load(f'./data/{model_type}{model_id}_0.npy', allow_pickle=True).item()
advs = data['advs']
classes = data['adv_class']
pert_lengths = data['pert_lengths']
dirs = data['dirs']
images = data['images']
labels = data['labels']


for batch_n in np.arange(1,n_batches):
    path = f'./data/{model_type}{model_id}_{batch_n}.npy'
    data = np.load(path, allow_pickle=True).item()
    advs = np.concatenate((advs, data['advs']), axis=0)
    classes = np.concatenate((classes, data['adv_class']), axis=0)
    pert_lengths = np.concatenate((pert_lengths, data['pert_lengths']), axis=0)
    dirs = np.concatenate((dirs, data['dirs']), axis=0)
    images = np.concatenate((images, data['images']), axis=0)
    labels = np.concatenate((labels, data['labels']), axis=0)

In [ ]:
pert_lengths[pert_lengths==0] = np.nan
sort_idx = np.argsort(pert_lengths,axis=-1)
for i, idx in enumerate(sort_idx):
    pert_lengths[i] = pert_lengths[i,idx]
    dirs[i] = dirs[i,idx]
    advs[i] = advs[i,idx]
    classes[i] = classes[i,idx]

In [ ]:
data = {
    'advs': advs,
    'dirs': dirs,
    'adv_class': classes,
    'pert_lengths': pert_lengths,
    'images': images,
    'labels': labels,
}
save_path = f'./data/concatenated/{model_type}_{model_id}.npy'
np.save(save_path, data)